### U Format, I Don't

Proof of concept demonstrating how to detect `'…'.format(…)` calls in Python source.

First, we need some stuff:

In [33]:
import ast
import fnmatch
import os

Where are we again?

In [34]:
pwd

u'/home/drocco/source/brightlink/brighttrac'

From here, we need a way to find all of the Python files that we'd like to check.

In [35]:
def all_pys():
    """Find all of the Python source files anywhere in the tree rooted at `.`"""
    
    for path, dirs, files in os.walk('.'):
        files = [os.path.join(path, file) 
                 for file in fnmatch.filter(files, '*.py')]
        for file in files:
            yield file

Then we need to check each one

In [44]:
def format_nanny(path):
    """Detect possibly bad `.format` calls in Python source.
    
    Given a Python source filename, find all of the calls to `.format` invoked
    on plain str objects (`''`) rather than Unicode objects (`u''`). For each,
    print out the path, line number, and string in question.
    
    """
    
    src = open(path).read()
    tree = ast.parse(src)

    # this is quick and dirty, the correct approach is to use a 
    # Visitor subclass 
    attrs = [node for node in ast.walk(tree) 
             if isinstance(node, ast.Attribute) and node.attr == 'format']

    for attr in attrs:
        try:
            _str = repr(attr.value.s)
            
            if _str[0] != 'u':
                print '{} {}: {}'.format(path, attr.lineno, _str)
        except:
            pass

#### Engage!

In [45]:
for py in all_pys():
    format_nanny(py)

./brighttrac2/model.py 62: '{}.model.model'
./brighttrac2/migrate/__init__.py 121: ' {}\n'
./brighttrac2/migrate/migrators/model.py 42: '{}_{}'
./brighttrac2/store/brighttrac_satchmo/utils.py 74: 'Invalid item, product, or slug: {0}'
./brighttrac2/store/brighttrac_satchmo/listeners.py 58: 'dummy-{0}'
./brighttrac2/store/brighttrac_satchmo/bt_job_queuer.py 67: 'Queuing job {}.{} to {} failed'
./brighttrac2/store/brighttrac_satchmo/views.py 62: 'Could not add item "{0}" to cart'
./brighttrac2/store/brighttrac_satchmo/views.py 136: 'No item found with ID {}.'
./brighttrac2/store/brighttrac_satchmo/views.py 558: 'Payment with transaction ID {} settled on {}<br/>'
./brighttrac2/store/brighttrac_satchmo/views.py 560: 'Payment with transaction ID {} not found, recording null settlement date'
./brighttrac2/store/brighttrac_satchmo/api/model.py 48: ' ({})'
./brighttrac2/store/brighttrac_satchmo/api/order.py 155: 'Transfered from User ({});'
./brighttrac2/store/brighttrac_satchmo/api/order.py 12

This won't detect certain dynamic cases, where `.format` is being applied to a variable, as here:

In [46]:
# %load -r 834-851 brighttrac2/base_model/renewal_audit.py
def _adjustment_requested(adjustment):
    if adjustment.auditor:
        template = 'Staff requested candidate adjustment for {what} for CE item ' \
                   '{title}'
    else:
        template = 'System-initiated candidate adjustment for {what} for CE item ' \
                   '{title}'

    items = [('credit_error', 'number of credits'),
             ('document_error', 'item documentation')]

    what = [description for attr, description in items
            if getattr(adjustment, attr)]

    note = template.format(what=', '.join(what),
                           title=adjustment.ce_item.title)

    adjustment.audit.note(note, staff=adjustment.auditor)